In [11]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Patch3D

from sim_functions import *

In [12]:
sccm_total = 15300
T_C = 650
T_STD = 298
T = T_C + T_STD
p = 100 #mbar
p_atm = 1013
flow_T = sccm_total * ((T)/ T_STD) * p_atm/p # cm3/60s
print("cm^3/min = ", flow_T)

# At entire chamber level:

cross_section = 13*4.5 # height * width, cm
flow_m_s = flow_T/cross_section/60 /100 #m/s
print("flow speed m/s = ", flow_m_s)

L = 1230 / cross_section /100 # length, meter
print("reactor length = ", L)

Diameter_hydraulic = 2*0.13*0.045/(0.13+0.045)
M = 2.016 *1e-3 #kg/mol
rho = (p*100) * M /8.31 /(T_C+T_STD)
mu = 0.192 * 1e-3 * 1e-1 # g/cm/s to Pa/s #https://pmc.ncbi.nlm.nih.gov/articles/PMC6728481/ # Dynamic viscosity
Re = rho * Diameter_hydraulic * flow_m_s / mu
print("Re = ", Re)

cm^3/min =  493052.25503355707
flow speed m/s =  1.4047072792978832
reactor length =  0.21025641025641026
Re =  12.517384246359603


In [24]:
D = 2.6e-3 # m^2/s # for TMIn

Pr = mu /rho /D
print("Prandtl = ", Pr)

Pe = Re * Pr
print("Peclet = ", Pe)

Sh = 0.332 * pow(Re,1/2) * pow(Pr, 1/3) #Sherwood (Nusselt number) # for laminar flat plate flow
print("Sh = ", Sh)

k_m = 2 * Sh * D/L # mass transfer coef
print("k_m = ", k_m)

del_boundary = 5.2 * L/ np.sqrt(Re)
print("del_boundary = ", del_boundary)

# set L to the growth area size
L_wafer = 3.8e-2

k_m = 2 * Sh * D / L_wafer # mass transfer coef
print("k_m = ", k_m)

Prandtl =  2.8856703296703303
Peclet =  36.12104432480272
Sh =  1.672285588338449
k_m =  0.04135847772134603
del_boundary =  0.30902655255139144
k_m =  0.22883908050947196


In [14]:
grow_rate = 180 * 1e-10 / 60# A/min to m/s
rho_InP = 4.81 #g/cm3 to kg/m3
M_InP = 145.8 #g/mol
J_InP = grow_rate * rho_InP /M_InP * 1e6 # cm to m conversion
print("J_InP = ", J_InP) # mol /m2 /s

P_TMIn = 0.46 # partial pressure, Pa
c_TMIn_stream = P_TMIn /8.31 /T # mol/m3
print("TMIn concentration = ", c_TMIn_stream)

k_reaction = J_InP/c_TMIn_stream
print("k_reaction = ", k_reaction)



J_InP =  9.897119341563784e-06
TMIn concentration =  5.839134386408526e-05
k_reaction =  0.169496344605475


In [15]:
def max_vx_no_clip_vy0(phi: float) -> float:
    """
    Max |vx| such that your D2Q9 equilibrium has no negative feq entries
    when vy = 0, using your special feq[:,0] expression.

    Returns 0.0 if clipping is unavoidable already at vx=0.
    """
    phi = float(phi)

    # If phi < 0 then moving populations with cu=0 are negative at u=0
    if phi <= 0.0:
        return 0.0

    # Your rest population feq0 is negative at u=0 if phi >= 9/5
    if phi >= 9.0/5.0:
        return 0.0

    # Bound from rest population feq[:,0]
    b_rest = np.sqrt((9.0 - 5.0*phi) / 6.0)

    # Bound from cx=0 populations (north/south) when vy=0
    b_cx0 = np.sqrt(2.0*phi / 3.0)

    # Bound from opposite-to-flow population (cx=-1) when vy=0
    if phi < 0.75:
        disc = 1.0 - 4.0*phi/3.0  # positive in this branch
        b_west = 0.5*(1.0 - np.sqrt(disc))  # small root (physical low-Mach bound)
    else:
        b_west = np.inf

    return float(min(b_rest, b_cx0, b_west))


In [44]:
# For a few millimeter layer above the 38mm long wafer
L_wafer = 0.038 # m, growth area length
L_height = 0.01 # m

ux = 2 # m/s, upper boundary
Pe = ux * L_wafer / D
print("Pe = ", Pe)

Re_real = rho * L_wafer * ux / mu
print("Re_real = ", Re_real)

Sc = Pe/Re_real
print("Sc = ", Sc)

Pe =  29.23076923076923
Re_real =  10.129628783378267
Sc =  2.8856703296703303


In [47]:
M_1 = 1
M_2 = 80*M_1

Bss = 1/np.sqrt(1/M_1 + 1/M_2)
B11 = 1/np.sqrt(1/M_1 + 1/M_1)

phi = Bss/B11/Sc
print("phi = ", phi)
print("vmax1 = ", max_vx_no_clip_vy0(phi))
phi_2 = phi * M_1/M_2
print("phi_2 = ", phi_2)
print("vmax2 = ", max_vx_no_clip_vy0(phi_2))

u_lbm = np.min([max_vx_no_clip_vy0(phi), max_vx_no_clip_vy0(phi_2)])
print("u_lbm = ", u_lbm)

u_set = 0.67 * u_lbm

number_grid = 400 # number of grid to represents 38m growth area

#delta_t = u_lbm * L_growth / ux
#delta_x = L_growth / number_grid
#D_lbm_require = D * delta_t/pow(delta_x, 2)

D_lbm_require = u_set * number_grid / Pe
print("D_lbm_require = ", D_lbm_require)

multiplier = 1 /Bss /D_lbm_require
print("multiplier = ", multiplier)


Bss = multiplier * 1/np.sqrt(1/M_1 + 1/M_2)
D_lbm = 1/Bss
print('Predict diffusion constant = ', D_lbm)

Pe_lbm = u_set * number_grid / D_lbm
print("Pe_lbm = ", Pe_lbm)


phi =  0.4870468824528469
vmax1 =  0.20394081810829717
phi_2 =  0.006088086030660586
vmax2 =  0.0020334971207603414
u_lbm =  0.0020334971207603414
D_lbm_require =  0.018643957812444815
multiplier =  53.97086820284735
Predict diffusion constant =  0.018643957812444815
Pe_lbm =  29.23076923076923


In [48]:
B11 = multiplier * 1/np.sqrt(1/M_1 + 1/M_1)
predict_lambda = (phi /3) * multiplier * 1/np.sqrt(1/M_1 + 1/M_1)
relaxation = predict_lambda/(1+predict_lambda/2)
print("relaxation = ", relaxation)
viscosity_kinematic = 1/3 * (1/relaxation - 0.5)
print('viscosity_kinematic = ',viscosity_kinematic)

print(1 /phi /B11)

Re_lbm = u_lbm *number_grid /viscosity_kinematic
print("Re_lbm = ", Re_lbm)

print(Re_real/Re_lbm)

relaxation =  1.5119421941205933
viscosity_kinematic =  0.05380031588699733
0.05380031588699735
Re_lbm =  15.118848930415332
0.6699999999999997


For 500 um region

In [22]:
# Focusing on

velocity_gradient = 200 # s^-1

#L_grid = L_growth/number_grid
#print(L_grid * 1e6, "micrometers")

L_sample = 500e-6 # 500 um
v_sample = velocity_gradient*L_sample
print("v_sample = ", velocity_gradient*L_sample, " m/s")

Pe_sample = v_sample * L_sample /D
print("Pe_sample = ", Pe_sample)

Re_sample = rho * L_sample * v_sample / mu
print("Re_sample = ", Re_sample)

Sc = Pe_sample/Re_sample
print("Sc = ", Sc)
#print("u_lbm = ", u_lbm)
u_set = u_lbm/1000
print("u_set = ", u_set)

n_grid_sample = 1000
D_lbm_require = u_set * n_grid_sample / Pe_sample
print("D_lbm_require = ", D_lbm_require)

Bss = 1/np.sqrt(1/M_1 + 1/M_2)
multiplier = 1 /Bss /D_lbm_require
print("multiplier = ", multiplier)

Bss = multiplier * 1/np.sqrt(1/M_1 + 1/M_2)
D_lbm = 1/Bss
print('Predict diffusion constant = ', D_lbm)

Pe_lbm = u_set * n_grid_sample / D_lbm
print("Pe_lbm = ", Pe_lbm)

B11 = multiplier * 1/np.sqrt(1/M_1 + 1/M_1)
predict_lambda = (phi /3) * multiplier * 1/np.sqrt(1/M_1 + 1/M_1)
relaxation = predict_lambda/(1+predict_lambda/2)
print("relaxation = ", relaxation)
viscosity_kinematic = 1/3 * (1/relaxation - 0.5)
print('viscosity_kinematic = ',viscosity_kinematic)

print(1 /phi /B11)

Re_lbm = u_set *n_grid_sample /viscosity_kinematic
print("Re_lbm = ", Re_lbm)

print(Re_sample/Re_lbm)

v_sample =  0.1  m/s
Pe_sample =  0.019230769230769232
Re_sample =  0.00666422946274886
Sc =  2.8856703296703303
u_set =  2.0334971207603415e-06
D_lbm_require =  0.10574185027953775
multiplier =  9.515916235765191
Predict diffusion constant =  0.10574185027953775
Pe_lbm =  0.019230769230769232
relaxation =  0.7065098867248728
viscosity_kinematic =  0.30513611995610435
0.3051361199561044
Re_lbm =  0.006664229462748862
0.9999999999999998


In [23]:
L_grid = L_sample/n_grid_sample
print("L_grid = ", L_grid*1e6, " um")

L_grid =  0.5  um
